In [ ]:
Install Required Libraries

In [ ]:
!pip install transformers torch adapter-transformers


In [ ]:
Import Required Libraries

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers.adapters import BertAdapterModel, AdapterConfig
from lora import LoRAConfig, apply_lora
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd


In [ ]:
Load Preprocessed Data

In [ ]:
# Load the preprocessed datasets
train_df = pd.read_csv('train_data.csv')
val_df = pd.read_csv('val_data.csv')
test_df = pd.read_csv('test_data.csv')

# Check the first few rows
print(train_df.head())


In [ ]:
Preprocess Data for BERT

In [ ]:
# Tokenization function for BERT
def preprocess_data(texts, tokenizer, max_length=128):
    return tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=max_length)

# Apply tokenization on the training, validation, and test data
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
X_train = preprocess_data(train_df['text'].tolist(), tokenizer)
y_train = torch.tensor(train_df['sentiment'].tolist())

X_val = preprocess_data(val_df['text'].tolist(), tokenizer)
y_val = torch.tensor(val_df['sentiment'].tolist())

X_test = preprocess_data(test_df['text'].tolist(), tokenizer)
y_test = torch.tensor(test_df['sentiment'].tolist())


In [ ]:
Adapter Fine-Tuning


In [ ]:
# Load BERT with Adapters
adapter_model = BertAdapterModel.from_pretrained('bert-large-uncased', num_labels=3)

# Add an adapter and set it to training mode
config = AdapterConfig(hidden_size=128, reduction_factor=16)  # Intermediate size of adapters: 128 perceptrons
adapter_model.add_adapter("sentiment_adapter", config=config)
adapter_model.train_adapter("sentiment_adapter")

# Set the adapter model to train mode
adapter_model.train()


In [ ]:
LoRA Fine-Tuning


In [ ]:
# Load BERT model and apply LoRA fine-tuning
lora_model = BertForSequenceClassification.from_pretrained('bert-large-uncased', num_labels=3)

# Apply LoRA to specific layers of BERT
lora_config = LoRAConfig(r=8)
apply_lora(lora_model, config=lora_config, target_modules=['encoder.layer'])

# Set the LoRA model to train mode
lora_model.train()


In [ ]:
Training Phase

In [ ]:
def train_model(model, optimizer, X_train, y_train, X_val, y_val, epochs=3):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(**X_train, labels=y_train)
        loss = outputs.loss
        loss.backward()
        
        # Update weights
        optimizer.step()
        
        print(f'Epoch {epoch+1}/{epochs} - Loss: {loss.item()}')

    # Evaluate on validation set
    model.eval()
    with torch.no_grad():
        outputs = model(**X_val)
        preds = torch.argmax(outputs.logits, dim=1)
        accuracy = accuracy_score(y_val, preds)
        report = classification_report(y_val, preds, target_names=['negative', 'neutral', 'positive'])
    
    return accuracy, report


In [ ]:
Optimizer Setup


In [ ]:
# Set optimizer for each model
adapter_optimizer = AdamW(adapter_model.parameters(), lr=1e-5)
lora_optimizer = AdamW(lora_model.parameters(), lr=1e-5)


In [ ]:
Train Adapter Model


In [ ]:
# Train the adapter model
adapter_acc, adapter_report = train_model(adapter_model, adapter_optimizer, X_train, y_train, X_val, y_val)

# Print results
print("Adapter Model Validation Accuracy:", adapter_acc)
print("Adapter Model Classification Report:\n", adapter_report)


In [ ]:
Train LoRA Model


In [ ]:
# Train the LoRA model
lora_acc, lora_report = train_model(lora_model, lora_optimizer, X_train, y_train, X_val, y_val)

# Print results
print("LoRA Model Validation Accuracy:", lora_acc)
print("LoRA Model Classification Report:\n", lora_report)


In [ ]:
Test Both Models on the Test Set

In [ ]:
def evaluate_model(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        outputs = model(**X_test)
        preds = torch.argmax(outputs.logits, dim=1)
        accuracy = accuracy_score(y_test, preds)
        report = classification_report(y_test, preds, target_names=['negative', 'neutral', 'positive'])
    
    return accuracy, report

# Test Adapter Model
adapter_test_acc, adapter_test_report = evaluate_model(adapter_model, X_test, y_test)
print("Adapter Model Test Accuracy:", adapter_test_acc)
print("Adapter Model Test Classification Report:\n", adapter_test_report)

# Test LoRA Model
lora_test_acc, lora_test_report = evaluate_model(lora_model, X_test, y_test)
print("LoRA Model Test Accuracy:", lora_test_acc)
print("LoRA Model Test Classification Report:\n", lora_test_report)
